In [28]:
#Act like you are apart of the CDC and want to investigate the tuberculosis rate in the US for the year 2018...
#We have tracked data in HTML format and want to run operations on the dataset

In [29]:
"""
Data Retreival

Imports: requests - package that enables you to pull data from a url
         BeautifulSoup [from bs4]  - package that enables you to parse and extract the data from the HTML format

Functions: Range() - gives a range vector from 0 to the input integer
	        len() - gives the number of elements of the input
            requests.get() - receive data from website
	    BeautfiulSoup.({html data}.content, “html.parser”) - parse HTML content

Methods : .find() - find the first selected piece of html data by tag
	     .find_all() -find the all the selected pieces of html data by tag
	     ~find_all().get_text()  - extract the actual content from the html piece
	     .append() - to append elements to a list  [good to utilize in for loops] 
"""

'\nData Retreival\n\nImports: requests - package that enables you to pull data from a url\n         BeautifulSoup [from bs4]  - package that enables you to parse and extract the data from the HTML format\n\nFunctions: Range() - gives a range vector from 0 to the input integer\n\t        len() - gives the number of elements of the input\n            requests.get() - receive data from website\n\t    BeautfiulSoup.({html data}.content, “html.parser”) - parse HTML content\n\nMethods : .find() - find the first selected piece of html data by tag\n\t     .find_all() -find the all the selected pieces of html data by tag\n\t     ~find_all().get_text()  - extract the actual content from the html piece\n\t     .append() - to append elements to a list  [good to utilize in for loops] \n'

In [30]:
from bs4 import BeautifulSoup
import requests

In [31]:
page = requests.get("https://data.cdc.gov/api/views/u3yt-gdfa/rows.xml?accessType=DOWNLOAD")
soup = BeautifulSoup(page.content, 'html.parser')

In [32]:
soup.find("reporting_area")

<reporting_area>UNITED STATES</reporting_area>

In [33]:
soup.find_all('reporting_area')[0].get_text()

'UNITED STATES'

In [34]:
##Create Variables in Python
###Only want to analyze the variables regarding 2018

In [35]:
Area = []

for i in range(len(soup.find_all('reporting_area'))):
    val = soup.find_all('reporting_area')[i].get_text()
    Area.append(val)
    

Year = []

for i in range(len(soup.find_all('mmwr_year'))):
    val = soup.find_all('mmwr_year')[i].get_text()
    Year.append(val)

Quarter = []

for i in range(len(soup.find_all('mmwr_quarter'))):
    val = soup.find_all('mmwr_quarter')[i].get_text()
    Quarter.append(val)

Quarter_Total = []

for i in range(len(soup.find_all('tuberculosis_current_quarter'))):
    val = soup.find_all('tuberculosis_current_quarter')[0].get_text()
    Quarter_Total.append(val)

Cum2018_Total = []

for i in range(len(soup.find_all('tuberculosis_cum_2018'))):
    val = soup.find_all('tuberculosis_cum_2018')[i].get_text()
    Cum2018_Total.append(val)

In [36]:
"""
Data Preparation

Imports : Pandas - package that gives all of us the data formats and operations we typically use when analyzing data

Functions : pd.DataFrame() - creates data frame of input
	       pd. Series() - creates column vector out of input [use to define dtype of interest

Methods :  .iloc[] - indexing method used to index in the same format as R for dataframes ---> 
                 [rows,columns]
"""

'\nData Preparation\n\nImports : Pandas - package that gives all of us the data formats and operations we typically use when analyzing data\n\nFunctions : pd.DataFrame() - creates data frame of input\n\t       pd. Series() - creates column vector out of input [use to define dtype of interest\n\nMethods :  .iloc[] - indexing method used to index in the same format as R for dataframes ---> \n                 [rows,columns]\n'

In [37]:
##Redefine the Tainted Variables that are mssing observations (Quarter_Total,Cum2018_Total)
##n = 335

In [38]:
len(Quarter_Total)

290

In [39]:
len(Cum2018_Total)
###they dropped values in the loop because they didn't have a values in the observation

317

In [40]:
Quarter_Total = []

for i in range(len(soup.find_all(['tuberculosis_current_quarter','tuberculosis_current_quarter_flag']))):
    val = soup.find_all(['tuberculosis_current_quarter','tuberculosis_current_quarter_flag'])[i].get_text()
    Quarter_Total.append(val)

Cum2018_Total = []

for i in range(len(soup.find_all(['tuberculosis_cum_2018','tuberculosis_cum_2018_flag']))):
    val = soup.find_all(['tuberculosis_cum_2018','tuberculosis_cum_2018_flag'])[i].get_text()
    Cum2018_Total.append(val)

In [41]:
##Replace hyphens with 0 for Quater_Total, and fix Cum2018_Total observations depending on Quarter_Total

In [42]:
import pandas as pd
r_data = pd.DataFrame({ "a" : Quarter_Total, "b" : Cum2018_Total})

In [43]:
for i in range(len(r_data)):
    if r_data.iloc[i,0] == "-" and r_data.iloc[i,1] == "-"  :
            r_data.iloc[i,0] = 0
            r_data.iloc[i,1] = 0
    elif r_data.iloc[i,0] == "-" and r_data.iloc[i,1] != "-"  :
            r_data.iloc[i,0] = 0
            r_data.iloc[i,1] = r_data.iloc[i,1]
            
###Keep in mind when you use .iloc() you all the rows switch to the object datatype

In [44]:
##Store all the data with a dictionary then apply it to a DataFrame

In [45]:
data = {'Area' : Area ,'Quarter' : Quarter,'Year' : Year,'Quarter_Total' : pd.Series(r_data.a).astype(int), 'Cum2018_Total' : pd.Series(r_data.b).astype(int)}
df = pd.DataFrame(data)
df.head()        

,Area,Quarter,Year,Quarter_Total,Cum2018_Total
0,UNITED STATES,1,2018,1216,1216
1,NEW ENGLAND,1,2018,57,57
2,MID. ATLANTIC,1,2018,271,271
3,NEW YORK CITY,1,2018,157,157
4,E.N. CENTRAL,1,2018,125,125


In [46]:
"""
Data Exploration

Functions : pd.qcut() - cut data into bins that equalize the counts in each bin (e.g. 100 obs →      qcut(,4) = 4 bins of 25 observations)

Methods : .groupby() - group the observations of your data by a specific variable
	     .mean() - find mean of grouped data
	     .std() - find all the std’s of your data…. Just playing actually the standard deviation
	     .describe() - find the mean,std, min, max, and IQR of Data
"""

'\nData Exploration\n\nFunctions : pd.qcut() - cut data into bins that equalize the counts in each bin (e.g. 100 obs →      qcut(,4) = 4 bins of 25 observations)\n\nMethods : .groupby() - group the observations of your data by a specific variable\n\t     .mean() - find mean of grouped data\n\t     .std() - find all the std’s of your data…. Just playing actually the standard deviation\n\t     .describe() - find the mean,std, min, max, and IQR of Data\n'

In [47]:
##Depending on your model and overall purpose of the DS project you want to analyze different things
##In this case I want to guage the metrics of Quarter_Total and the Cumilative Totals

In [48]:
df.Quarter_Total.mean()

70.48358208955224

In [49]:
df.Quarter_Total.std()

206.6214947553004

In [50]:
df.groupby("Area")["Quarter_Total"].mean()

Area
ALABAMA             21.2
ALASKA               9.6
AMER. SAMOA          0.2
ARIZONA             13.0
ARKANSAS             8.8
C.N.M.I.             1.8
CALIFORNIA         377.6
COLORADO            13.8
CONNECTICUT          9.8
DELAWARE             1.6
DIST. OF COL.        7.6
E.N. CENTRAL       171.6
E.S. CENTRAL        88.2
FLORIDA             99.8
GEORGIA             54.2
GUAM                 9.4
HAWAII              17.6
IDAHO                3.2
ILLINOIS            75.6
INDIANA             32.6
IOWA                 4.0
KANSAS               0.4
KENTUCKY            14.4
LOUISIANA           17.4
MAINE                3.4
MARYLAND            40.8
MASSACHUSETTS       38.0
MICHIGAN            14.2
MID. ATLANTIC      295.0
MINNESOTA           40.6
                   ...  
NEW HAMPSHIRE        0.2
NEW JERSEY          57.2
NEW MEXICO           6.6
NEW YORK            48.6
NEW YORK CITY      140.0
NORTH CAROLINA      37.0
NORTH DAKOTA         2.4
OHIO                43.8
OKLAHOMA            

In [51]:
df.groupby("Quarter")["Quarter_Total"].mean()

Quarter
1    54.597015
2    69.507463
3    63.805970
4    82.253731
Name: Quarter_Total, dtype: float64

In [52]:
df.groupby("Quarter")["Cum2018_Total"].mean()

Quarter
1     54.597015
2    143.910448
3    237.238806
4    362.850746
Name: Cum2018_Total, dtype: float64

In [53]:
quartiles = pd.qcut(df.Quarter_Total,5)
grouped = df.Quarter_Total.groupby(quartiles)
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
Quarter_Total,,,,,,,,
"(-0.001, 2.0]",77.0,0.636364,0.825723,0.0,0.00,0.0,1.00,2.0
"(2.0, 10.0]",62.0,6.064516,2.401304,3.0,4.00,6.0,8.00,10.0
"(10.0, 25.8]",62.0,17.725806,4.093855,11.0,15.00,17.0,21.00,25.0
"(25.8, 60.0]",70.0,42.900000,9.655141,27.0,33.25,44.0,51.00,60.0
"(60.0, 1828.0]",64.0,298.203125,399.877925,61.0,81.25,148.5,319.25,1828.0


In [54]:
"""
Each data has different complications for retrieving and preparing the data. The dataset has unique analysis for exploration as well.

Every dataset will go through the DS method (Reteival,Preparation,Exploration, then Visualization/Modeling) but each instance is different and will never quite be uniform

This is why a creative sense while being a Data Scientist is very impotant 

Here are some links to datasets for you guys to try for you guys to try these skills on :

1. [HTML Example] Here is dataset measuring tobacco use of various forms which each observations being its own survey. 

Link:
https://chronicdata.cdc.gov/views/tbfm-vbpp/rows.xml?accessType=DOWNLOAD

(Documentation : https://catalog.data.gov/dataset/national-adult-tobacco-survey-nats-51225)


2. [CSV Example] Here is a link to a page containing the csv file of a dataset of popular baby names in the New York Area

Link:
https://catalog.data.gov/dataset/most-popular-baby-names-by-sex-and-mothers-ethnic-group-new-york-city-8c742
"""

'\nEach data has different complications for retrieving and preparing the data. The dataset has unique analysis for exploration as well.\n\nEvery dataset will go through the DS method (Reteival,Preparation,Exploration, then Visualization/Modeling) but each instance is different and will never quite be uniform\n\nThis is why a creative sense while being a Data Scientist is very impotant \n\nHere are some links to datasets for you guys to try for you guys to try these skills on :\n\n1. [HTML Example] Here is dataset measuring tobacco use of various forms which each observations being its own survey. \n\nLink:\nhttps://chronicdata.cdc.gov/views/tbfm-vbpp/rows.xml?accessType=DOWNLOAD\n\n(Documentation : https://catalog.data.gov/dataset/national-adult-tobacco-survey-nats-51225)\n\n\n2. [CSV Example] Here is a link to a page containing the csv file of a dataset of popular baby names in the New York Area\n\nLink:\nhttps://catalog.data.gov/dataset/most-popular-baby-names-by-sex-and-mothers-eth